In [ ]:
import numpy as np
import imageio.v3 as iio
import os
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

In [ ]:
def psnr(gt, est, max_pixel): 
    mse = np.mean((gt - est) ** 2) 
    max_pixel = max_pixel
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse)) 
    return psnr

def nrmse_np(x,y):
    y = y / np.linalg.norm(y)
    y = y * np.linalg.norm(x)
    num = np.linalg.norm(x-y)
    denom = np.linalg.norm(x)
    return num/denom

def mse_np(x,y):
    y = y / np.linalg.norm(y)
    x = x / np.linalg.norm(x)
    num = np.linalg.norm(x-y)
    return num**2

def nrmse(x, y):
    num = torch.norm(x-y, p=2)
    denom = torch.norm(x,p=2)
    return num/denom

In [ ]:
gt_list = sorted(os.listdir("/home/asad/ambient-diffusion/data/00000/"))[1:37]

In [ ]:
ambient_nrmse_master = []
sup_nrmse_master = []

ambient_mse_master = []
sup_mse_master = []

ambient_nrmse_std = []
sup_nrmse_std = []

ambient_mse_std = []
sup_mse_std = []

corruption = "inpainting"
noisy = ""
corr = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for c in corr:
    ambient_nrmse = []
    sup_nrmse = []
    ambient_mse = []
    sup_mse = []
    
    ambient_list = sorted(os.listdir("/home/asad/ambient-diffusion/results/" + str(corruption) + "/ambient/ffhq_ambient" + str(noisy) + "-op_corr" + str(c) + "/185083/"))
    sup_list = sorted(os.listdir("/home/asad/ambient-diffusion/results/" + str(corruption) + "/supervised/ffhq_supervised" + str(noisy) + "-op_corr" + str(c) + "/185083/"))
    if ".ipynb_checkpoints" in ambient_list:
        ambient_list.remove(".ipynb_checkpoints")
    if ".ipynb_checkpoints" in sup_list:    
        sup_list.remove(".ipynb_checkpoints")
    for i in range(36):
        ambient = iio.imread("/home/asad/ambient-diffusion/results/" + str(corruption) + "/ambient/ffhq_ambient" + str(noisy) + "-op_corr" + str(c) + "/185083/" + ambient_list[i])
        sup = iio.imread("/home/asad/ambient-diffusion/results/" + str(corruption) + "/supervised/ffhq_supervised" + str(noisy) + "-op_corr" + str(c) + "/185083/" + sup_list[i])
        gt = iio.imread("/home/asad/ambient-diffusion/data/00000/" + str(gt_list[i]))
        ambient_nrmse.append(nrmse_np(gt, ambient))
        sup_nrmse.append(nrmse_np(gt, sup))
        
        ambient_mse.append(mse_np(gt, ambient))
        sup_mse.append(mse_np(gt, sup))
    
    ambient_nrmse_master.append(np.mean(ambient_nrmse))
    sup_nrmse_master.append(np.mean(sup_nrmse))

    ambient_mse_master.append(np.mean(ambient_mse))
    sup_mse_master.append(np.mean(sup_mse))
    
    ambient_nrmse_std.append(np.std(ambient_nrmse))
    sup_nrmse_std.append(np.std(sup_nrmse))

    ambient_mse_std.append(np.std(ambient_mse))
    sup_mse_std.append(np.std(sup_mse))

In [ ]:
ambient_nrmse_master = np.array(ambient_nrmse_master)
sup_nrmse_master = np.array(sup_nrmse_master)

df = pd.DataFrame(np.array(corr)*10, columns=["Downsampling Factor"])
df["FS-DPS"] = sup_nrmse_master
df["A-DPS"] = ambient_nrmse_master
df = df.set_index(df.columns[0])

ambient_mse_master = np.array(ambient_mse_master)
sup_mse_master = np.array(sup_mse_master)

df_mse = pd.DataFrame(np.array(corr)*10, columns=["Downsampling Factor"])
df_mse["FS-DPS"] = sup_mse_master
df_mse["A-DPS"] = ambient_mse_master
df_mse = df_mse.set_index(df_mse.columns[0])

ambient_nrmse_std = np.array(ambient_nrmse_std)
sup_nrmse_std = np.array(sup_nrmse_std)

df_std = pd.DataFrame(np.array(corr)*10, columns=["Downsampling Factor"])
df_std["FS-DPS"] = sup_nrmse_std
df_std["A-DPS"] = ambient_nrmse_std
df_std = df_std.set_index(df_std.columns[0])

ambient_mse_std = np.array(ambient_mse_std)
sup_mse_std = np.array(sup_mse_std)

df_mse_std = pd.DataFrame(np.array(corr)*10, columns=["Downsampling Factor"])
df_mse_std["FS-DPS"] = sup_mse_std
df_mse_std["A-DPS"] = ambient_mse_std
df_mse_std = df_mse_std.set_index(df_mse_std.columns[0])

In [ ]:
std_values_1 = df_mse_std["FS-DPS"].to_numpy()
std_values_2 = df_mse_std["A-DPS"].to_numpy()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(df_mse["FS-DPS"], linestyle='--', marker="o")
plt.plot(df_mse["A-DPS"], linestyle='--', marker="^", color="red")
plt.fill_between(np.array(corr)*10, df_mse["FS-DPS"] + std_values_1, df_mse["FS-DPS"] - std_values_1, alpha=0.3)  # For the first line
plt.fill_between(np.array(corr)*10, df_mse["A-DPS"] + std_values_2, df_mse["A-DPS"] - std_values_2, color="red", alpha=0.3)  # For the second line

plt.legend(["p=0.0", "p=0.6"])
plt.xticks(np.array(corr))
plt.title("Box Inpainting - MSE results on FFHQ")
plt.ylabel("MSE")
plt.xlabel("Box Scaling Factor")
plt.xticks([10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0])
plt.grid(color='lightgrey', linewidth=0.5)
plt.gca().invert_xaxis()